In [1]:
%%capture
%pip install transformers datasets accelerate torch evaluate bert_score rouge_score bitsandbytes

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import evaluate

# Load data
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df['abstract'].tolist(), df['title'].tolist()

train_abstracts, train_titles = load_data("/kaggle/input/springer-journal-final/train.csv")
val_abstracts, val_titles = load_data("/kaggle/input/springer-journal-final/val.csv")
test_abstracts, test_titles = load_data("/kaggle/input/springer-journal-final/test.csv")

# Tokenization
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_abstracts + train_titles)

X_train = pad_sequences(tokenizer.texts_to_sequences(train_abstracts), padding='post')
y_train = np.array([seq[0] for seq in tokenizer.texts_to_sequences(train_titles) if len(seq) > 0])

X_val = pad_sequences(tokenizer.texts_to_sequences(val_abstracts), padding='post')
y_val = np.array([seq[0] for seq in tokenizer.texts_to_sequences(val_titles) if len(seq) > 0])

# Load GloVe embeddings
glove_path = "/kaggle/input/glove6b100dtxt/glove.6B.100d.txt"
embedding_dim = 100
embedding_index = {}

with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

word_index = tokenizer.word_index
max_vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((max_vocab_size, embedding_dim))

for word, i in word_index.items():
    vector = embedding_index.get(word)
    if vector is not None:
        embedding_matrix[i] = vector

# Build LSTM model
model = Sequential([
    Embedding(max_vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False),
    LSTM(128),
    Dense(max_vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Generate predictions
X_test = pad_sequences(tokenizer.texts_to_sequences(test_abstracts), padding='post')
predicted_tokens = np.argmax(model.predict(X_test), axis=1)
predicted_titles = [tokenizer.index_word.get(token, "<UNK>") for token in predicted_tokens]

# Evaluate model
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

rouge_scores = rouge.compute(predictions=predicted_titles, references=test_titles)
bert_scores = bertscore.compute(predictions=predicted_titles, references=test_titles, lang="en")

# Print results
print("ROUGE:", rouge_scores)
print("BERTScore (averaged):")
print("  Precision:", sum(bert_scores["precision"]) / len(bert_scores["precision"]))
print("  Recall:", sum(bert_scores["recall"]) / len(bert_scores["recall"]))
print("  F1:", sum(bert_scores["f1"]) / len(bert_scores["f1"]))

Epoch 1/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 90s 60ms/step - accuracy: 0.1132 - loss: 8.0754 - val_accuracy: 0.1151 - val_loss: 7.2815
Epoch 2/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 86s 60ms/step - accuracy: 0.1121 - loss: 6.8667 - val_accuracy: 0.1151 - val_loss: 7.3665
Epoch 3/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 86s 60ms/step - accuracy: 0.1129 - loss: 6.7641 - val_accuracy: 0.1151 - val_loss: 7.4687
Epoch 4/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 86s 60ms/step - accuracy: 0.1135 - loss: 6.7418 - val_accuracy: 0.1151 - val_loss: 7.5226
Epoch 5/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 85s 60ms/step - accuracy: 0.1146 - loss: 6.7363 - val_accuracy: 0.1151 - val_loss: 7.5731
Epoch 6/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 85s 60ms/step - accuracy: 0.1125 - loss: 6.7492 - val_accuracy: 0.1151 - val_loss: 7.6047
Epoch 7/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 85s 59ms/step - accuracy: 0.1138 - loss: 6.7449 - val_accuracy: 0.1151 - val_loss: 7.6485
Epoch 8/10
1426/1426 ━━━━━━━━━━━━━━━━━━━━ 86s 60ms/step - accuracy: 0.1134 -

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE: {'rouge1': 0.03709921634625642, 'rouge2': 0.0, 'rougeL': 0.0370465399996594, 'rougeLsum': 0.037128791069575444}
BERTScore (averaged):
  Precision: 0.837819782197946
  Recall: 0.7814797677610699
  F1: 0.8085472839816665
